In [ ]:
%%capture
! pip install torchmetrics
! pip install wandb -Uq
! pip install timm
! pip install onnxruntime-gpu

In [ ]:
import onnxruntime

# clone repository

In [ ]:
! git clone https://github.com/guillepinto/Fine-Tuned-Thermal-Breast-Cancer-Diagnosis.git
%cd Fine-Tuned-Thermal-Breast-Cancer-Diagnosis

## initial setup to connect to Kaggle

In [ ]:
%%capture
! pip install kaggle

In [ ]:
from google.colab import files
files.upload() # upload your key

In [ ]:
! mkdir ~/kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

## dataset loading

Someone uploaded the dataset to kaggle, so we'll take advantage of it.

In [ ]:
%%capture
! kaggle datasets download -d asdeepak/thermal-images-for-breast-cancer-diagnosis-dmrir
! unzip thermal-images-for-breast-cancer-diagnosis-dmrir.zip

## make inference

In [ ]:
execfile('make_dataset.py')
execfile('test.py')

In [ ]:
# Función de inferencia ONNX
def onnx_inference(onnx_path, test_dataset):
    session = onnxruntime.InferenceSession(onnx_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    input_name = session.get_inputs()[0].name
    
    all_preds = []
    all_labels = []

    for img, label in test_dataset:
        ort_inputs = {input_name: img[np.newaxis, ...].astype(np.float32)}
        ort_outputs = session.run(None, ort_inputs)
        pred = ort_outputs[0]
        all_preds.append(pred[0][0])
        all_labels.append(label)

    return np.array(all_preds), np.array(all_labels)

In [ ]:
for fold, onnx_path in enumerate(onnx_models, 1):
        print(f"Evaluando modelo del fold {fold}")
        test_dataset = get_data(fold=fold, n_channels=1)
        preds, labels = onnx_inference(onnx_path, test_dataset)
        accuracy, f1, recall, precision = calculate_metrics(preds, labels)
        print(f"Accuracy: {accuracy:.3f}, F1: {f1:.3f}, Recall: {recall:.3f}, Precision: {precision:.3f}")